# Примеры заданий

Задания на экзамене будут аналогичны индивидуальным заданиям курса и нижеприведенным примерам.

Количество заданий, количество попыток на каждое отдельное задания, количество баллов за задание могут отличаться от примера.

## Задача 1

Уравнение линейной регрессии задается соотношением $2 + 3 X_1 - 4 X_2 = 0$.

Предскажите отклик $X_2$ для тестового объекта — (4):

In [1]:
print((2+3*4)/4)

3.5


Может ли данное уравнение задавать регрессию, построенную по тренировочным данным: $(1, 2)$ и $(-1, 0)$? Если нет, постройте правильное уравнение и введите его в поле ввода, если да — введите $1$.

In [2]:
import numpy as np
from sklearn.linear_model import LinearRegression

X = np.array([[1], [-1]])
y = np.array([2, 0])
reg = LinearRegression().fit(X, y)
print(f'X1 * {reg.coef_[0]} - X2 + {reg.intercept_} = 0')

X1 * 0.9999999999999998 - X2 + 1.0 = 0


## Задача 2

Модель логистической регрессии обучена на определение класса +. $\Psi = -2.992 + 0.240 X_1 + 1.102 X_2$. На рисунке отображены точки, на которых обучена модель, но вам неизвестно какие из них относятся к классу + и -.

Определите, в каком направлении расположен нормальный вектор гиперплоскости.

In [36]:
norm_vec = np.array([0.240, 1.102])
print('[X1, X2] =', norm_vec)

[X1, X2] = [0.24  1.102]


Учитывая, что тестовая точка $A$ с координатами $(0, 0)$ относится к классу +, верно ли она классифицирована моделью?

In [37]:
def psi(x1, x2):
  return - 2.992 + 0.240 * x1 + 1.102 * x2

print(psi(0, 0))  # class '-'

-2.992


Вычислите вероятность отнесения точки $A$ к классу +:

In [38]:
print(np.round(1 / (1 + np.exp(-psi(0, 0))), 3))

0.048


## Задача 3

В данном упражнении вам предстоит решить задачу многокласовой классификации, используя в качестве тренировочного набора данных — набор данных MNIST, содержащий образы рукописных цифр.

1. Используйте метод главных компонент для набора данных MNIST (train dataset объема 60000). Определите, какое минимальное количество главных компонент $M$ необходимо использовать, чтобы доля объясненной дисперсии превышала 0.83 (была строго больше указанного значения).

In [39]:
import numpy as np
from sklearn.decomposition import PCA
from keras.datasets import mnist
from sklearn.model_selection import train_test_split

(X_train, y_train), (X_pred, y_pred) = mnist.load_data()

X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_pred = np.reshape(X_pred, (X_pred.shape[0], -1))

pca = PCA(n_components=52, svd_solver='full')
pca.fit(X_train)
print(sum(pca.explained_variance_ratio_))  # M = 52

0.8309650832314033


2. Найдите счеты, отвечающие найденным в предыдущем пункте $M$ главным компонент `PCA(n_components=M, svd_solver='full')`.

In [40]:
X_centered = X_train - X_train.mean(axis=0)
weights = pca.components_.T
print(X_centered.shape, weights.shape)
z = X_centered @ weights
print(z.shape)

(60000, 784) (784, 52)
(60000, 52)


3. Разделите полученную выборку (после сокращения размерности) случайным образом в отношении 70/30: train_test_split(X_train, y_train, test_size=0.3, random_state=126). Введите выборочное среднее нулевой колонки для полученного тренировочного набора:

In [41]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(z, y_train, test_size=0.3, random_state=126)
print(np.mean(X_train1[:, 0]).round(3))

0.597


4. Используя метод многоклассовой классификации One-vs-All `OneVsRestClassifier()`, обучите алгоритм случайного леса `RandomForestClassifier()` с параметрами `criterion='gini'`, `min_samples_leaf=10`, `max_depth=20`, `n_estimators=10`, `random_state=126`. Выполните оценку с помощью тестовых данных.

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

rfc = RandomForestClassifier(criterion='gini',
                             min_samples_leaf=10,
                             max_depth=20,
                             n_estimators=10,
                             random_state=126)
ovr = OneVsRestClassifier(rfc).fit(X_train1, y_train1)
y_pred1 = ovr.predict(X_test1)

target_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
print(classification_report(y_test1, y_pred1, target_names=target_names))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95      1817
           1       0.96      0.97      0.97      2048
           2       0.91      0.90      0.91      1784
           3       0.89      0.89      0.89      1785
           4       0.92      0.93      0.92      1745
           5       0.89      0.88      0.88      1610
           6       0.94      0.96      0.95      1736
           7       0.94      0.93      0.93      1901
           8       0.89      0.86      0.88      1804
           9       0.89      0.88      0.89      1770

    accuracy                           0.92     18000
   macro avg       0.92      0.92      0.92     18000
weighted avg       0.92      0.92      0.92     18000



Введите количество верно классифицированных объектов класса 5 для тестовых данных:

In [18]:
correct_5s = 0
for id, i in enumerate(y_pred1):
  if i == y_test1[id] == 5:
    correct_5s += 1

print(correct_5s)

1414


5. Примените полученное ранее преобразование метода главных компонент к новым данным (идентификаторы строк соответствуют файлам изображений).

In [30]:
import pandas as pd
df = pd.read_csv('pred_for_task.csv')

In [31]:
fnames = df['FileName']
y = np.array(df['Label'])
X = np.array(df.drop(['FileName', 'Label'], axis=1))
print(y.shape, X.shape)
X = pca.transform(X)

(30,) (30, 784)


6. Выполните предсказание для указанных изображений цифр с помощью обученных алгоритмов.

Введите вероятность отнесения изображения file20 к назначенному классу алгоритмом случайного леса.

In [43]:
y_pred2 = ovr.predict_proba(X)
print(f'P(class({fnames[19]}) == 4) =', round(y_pred2[19][4], 3))

P(class(file20) == 4) = 0.963
